In [1]:
from tensorflow.keras import backend, layers, metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import shutil
import time
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import preprocess_input
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# prepare the dataset, iterate over each folder (personA : personB) in each folder there is test and train, read csv in each one and split images into two folders, real & fake
def prepare_dataset(base_path):
    for folder in os.listdir(base_path):
        path = os.path.join(base_path, folder)
        if os.path.isdir(path):
            train_path = os.path.join(path, 'train')
            test_path = os.path.join(path, 'test')
            # Create real and forged folders in train path
            os.makedirs(os.path.join(train_path, 'real'), exist_ok=True)
            os.makedirs(os.path.join(train_path, 'forged'), exist_ok=True)
            for file in os.listdir(train_path):
                if file.endswith('.csv'): 
                    df = pd.read_csv(os.path.join(train_path, file))
                    for index, row in df.iterrows():
                        img_path = os.path.join(train_path, row['image_name'])
                        if os.path.exists(img_path):
                            if row['label'] == 'real':
                                shutil.copy(img_path, os.path.join(train_path, 'real', row['image_name']))
                            else:
                                shutil.copy(img_path, os.path.join(train_path, 'forged', row['image_name']))
                        else:
                            print(f"Image {img_path} does not exist.")
                else:
                    # If the file is not a CSV, skip it
                    continue
            # Create real and forged folders in test path
            os.makedirs(os.path.join(test_path, 'real'), exist_ok=True)
            os.makedirs(os.path.join(test_path, 'forged'), exist_ok=True)
            for file in os.listdir(test_path):
                if file.endswith('.csv'):
                    df = pd.read_csv(os.path.join(test_path, file))
                    for index, row in df.iterrows():
                        img_path = os.path.join(test_path, row['image_name'])
                        if os.path.exists(img_path):
                            if row['label'] == 'real':
                                shutil.copy(img_path, os.path.join(test_path, 'real', row['image_name']))
                            else:
                                shutil.copy(img_path, os.path.join(test_path, 'forged', row['image_name']))
                        else:
                            print(f"Image {img_path} does not exist.")
                else:
                    # If the file is not a CSV, skip it
                    continue
# Load and preprocess the dataset
prepare_dataset("/Users/mohamedkorayem/Library/Mobile Documents/com~apple~CloudDocs/ITI/21-Computer_Vision/Siamese_network/Siamese")


In [3]:
# create a function to load train triplets, create all possible triplets, and append them to a list,
# ensuring anchor, positive, and negative images are different
# number of triplets = number of forged images * (number of real C 2)
def load_triplets(base_path,set_type):
    triplets = []
    for folder in os.listdir(base_path):
        path = os.path.join(base_path, folder)
        if os.path.isdir(path):
            set_path = os.path.join(path, set_type)
            real_path = os.path.join(set_path, 'real')
            forged_path = os.path.join(set_path, 'forged')
            real_images = os.listdir(real_path)
            forged_images = os.listdir(forged_path)
            # Create all possible triplets
            for i in range(len(real_images)):
                for j in range(len(real_images)):
                    for k in range(len(forged_images)):
                        if j>i:
                            triplet = (os.path.join(real_path, real_images[i]), os.path.join(real_path, real_images[j]), os.path.join(forged_path, forged_images[k]))
                            triplets.append(triplet)
    return triplets
# Load the dataset
train_triplets = load_triplets("/Users/mohamedkorayem/Library/Mobile Documents/com~apple~CloudDocs/ITI/21-Computer_Vision/Siamese_network/Siamese", "train")
test_triplets = load_triplets("/Users/mohamedkorayem/Library/Mobile Documents/com~apple~CloudDocs/ITI/21-Computer_Vision/Siamese_network/Siamese", "test")
print(f"Number of training triplets: {len(train_triplets)}")
print(f"Number of testing triplets: {len(test_triplets)}")
            

Number of training triplets: 19000
Number of testing triplets: 120


In [4]:
def read_image(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (128, 128 ))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image
# patch generator
def get_batch(triplet_list, batch_size=256, preprocess=False):
    batch_steps = len(triplet_list)//batch_size
    for i in range(batch_steps+1):
        anchor   = []
        positive = []
        negative = []
        
        j = i*batch_size
        while j<(i+1)*batch_size and j<len(triplet_list):
            a, p, n = triplet_list[j]
            anchor.append(read_image(a))
            positive.append(read_image(p))
            negative.append(read_image(n))
            j+=1
            
        anchor = np.array(anchor)
        positive = np.array(positive)
        negative = np.array(negative)
        
        if preprocess:
            anchor = preprocess_input(anchor)
            positive = preprocess_input(positive)
            negative = preprocess_input(negative)
        
        yield ([anchor, positive, negative])

In [5]:
def get_encoder(input_shape):
    """ Returns the image encoding model """

    pretrained_model = Xception(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg',
    )
    
    for i in range(len(pretrained_model.layers)-27):
        pretrained_model.layers[i].trainable = False

    encode_model = Sequential([
        pretrained_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(256, activation="relu"),
        layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name="Encode_Model")
    return encode_model

In [6]:
class DistanceLayer(layers.Layer):
    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)
    

def get_siamese_network(input_shape = (128, 128, 3)):
    encoder = get_encoder(input_shape)
    
    # Input Layers for the images
    anchor_input   = layers.Input(input_shape, name="Anchor_Input")
    positive_input = layers.Input(input_shape, name="Positive_Input")
    negative_input = layers.Input(input_shape, name="Negative_Input")
    
    ## Generate the encodings (feature vectors) for the images
    encoded_a = encoder(anchor_input)
    encoded_p = encoder(positive_input)
    encoded_n = encoder(negative_input)
    
    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    distances = DistanceLayer()(
        encoder(anchor_input),
        encoder(positive_input),
        encoder(negative_input)
    )
    
    # Creating the Model
    siamese_network = Model(
        inputs  = [anchor_input, positive_input, negative_input],
        outputs = distances,
        name = "Siamese_Network"
    )
    return siamese_network

siamese_network = get_siamese_network()
siamese_network.summary()

I0000 00:00:1744118748.755542  929020 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1744118748.755732  929020 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "Siamese_Network"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Anchor_Input        │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Positive_Input      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Negative_Input      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Encode_Model        │ (None, 256)       │ 22,043,944 │ Anchor_Input[0][… │
│ (Sequential)        │                   │            │ Positive_Input[0… │
│                     │                   │            │ Negative_Input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ distance_layer      │ [(None), (None)]  │          0 │ Encode_Model[3][… │
│ (DistanceLayer)     │                   │            │ Encode_Model[4][… │
│                     │                   │            │ Encode_Model[5][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 22,043,944 (84.09 MB)

 Trainable params: 9,583,800 (36.56 MB)

 Non-trainable params: 12,460,144 (47.53 MB)

In [7]:
class SiameseModel(Model):
    # Builds a Siamese model based on a base-model
    def __init__(self, siamese_network, margin=1.0):
        super(SiameseModel, self).__init__()
        
        self.margin = margin
        self.siamese_network = siamese_network
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape get the gradients when we compute loss, and uses them to update the weights
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)
            
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.siamese_network.trainable_weights))
        
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)
        
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # Get the two distances from the network, then compute the triplet loss
        ap_distance, an_distance = self.siamese_network(data)
        loss = tf.maximum(ap_distance - an_distance + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics so the reset_states() can be called automatically.
        return [self.loss_tracker]

In [8]:
siamese_model = SiameseModel(siamese_network)
optimizer = Adam(learning_rate=1e-3, epsilon=1e-01)
siamese_model.compile(optimizer=optimizer)

In [9]:
def test_on_triplets(batch_size = 256):
    pos_scores, neg_scores = [], []

    for data in get_batch(test_triplets, batch_size=batch_size):
        prediction = siamese_model.predict(data)
        pos_scores += list(prediction[0])
        neg_scores += list(prediction[1])
    
    accuracy = np.sum(np.array(pos_scores) < np.array(neg_scores)) / len(pos_scores)
    ap_mean = np.mean(pos_scores)
    an_mean = np.mean(neg_scores)
    ap_stds = np.std(pos_scores)
    an_stds = np.std(neg_scores)
    
    print(f"Accuracy on test = {accuracy:.5f}")
    return (accuracy, ap_mean, an_mean, ap_stds, an_stds)

In [10]:
save_all = False
epochs = 30
batch_size = 32

max_acc = 0
train_loss = []
test_metrics = []

for epoch in range(1, epochs+1):
    t = time.time()
    
    # Training the model on train data
    epoch_loss = []
    for data in get_batch(train_triplets, batch_size=batch_size):
        loss = siamese_model.train_step(data)
        epoch_loss.append(loss)
    epoch_loss = sum(epoch_loss)/len(epoch_loss)
    train_loss.append(epoch_loss)

    print(f"\nEPOCH: {epoch} \t (Epoch done in {int(time.time()-t)} sec)")
    print(f"Loss on train    = {epoch_loss:.5f}")
    
    # Testing the model on test data
    metric = test_on_triplets(batch_size=batch_size)
    test_metrics.append(metric)
    accuracy = metric[0]
    
    # Saving the model weights
    if save_all or accuracy>=max_acc:
        siamese_model.save_weights("siamese_model")
        max_acc = accuracy

# Saving the model after all epochs run
siamese_model.save_weights("siamese_model-final")

KeyboardInterrupt: 